In [2]:
!mkdir -p data


In [3]:
%%writefile data/teams_venues_times.py
teams = [
    "Al Ahly", "Zamalek", "Pyramids", "Masry", "Future", "Ismaily",
    "Smouha", "ENPPI", "Ceramica", "National Bank", "Talaea El Gaish",
    "Alexandria Union", "El Dakhleya", "El Gouna", "Zed",
    "Modern Sport", "Pharco", "Wadi Degla"
]

venues = [
    "Cairo Stadium", "Borg El Arab", "Air Defense Stadium",
    "Suez Stadium", "Alexandria Stadium", "Petro Sport Stadium",
    "Military Academy Stadium", "Al Salam Stadium", "El Sekka El Hadeed Stadium",
    "Zed Club Stadium"
]

match_times = ["17:00", "20:00"]


Writing data/teams_venues_times.py


In [4]:
import random
from datetime import datetime, timedelta
from data.teams_venues_times import teams, venues, match_times

# مدة البطولة
start_date = datetime(2025, 5, 1)
end_date = datetime(2026, 1, 31)

# إعداد أيام البطولة
all_dates = []
current_date = start_date
while current_date <= end_date:
    all_dates.append(current_date)
    current_date += timedelta(days=1)


# تعريف الـ Match
class Match:
    def __init__(self, team1, team2, date, time, venue):
        self.team1 = team1
        self.team2 = team2
        self.date = date
        self.time = time
        self.venue = venue

    def __repr__(self):
        return f"{self.team1} vs {self.team2} on {self.date.strftime('%Y-%m-%d')} at {self.time} in {self.venue}"


# توليد جدول دوري أسبوعي مع تكرار المباريات مرتين
def generate_weekly_schedule(teams, venues, all_dates, match_times, min_rest_days=4):
    schedule = []
    last_played = {team: start_date - timedelta(days=min_rest_days) for team in teams}

    # معلومات لكل يوم
    day_info = {d: {'count': 0, 'venues': set(), 'times': set()} for d in all_dates}

    pairings = []
    for t1 in teams:
        for t2 in teams:
            if t1 != t2:
                pairings.append((t1, t2))
                pairings.append((t2, t1))

    random.shuffle(pairings)
    matches_count = {(t1, t2): 0 for t1 in teams for t2 in teams if t1 != t2}

    week_dates = all_dates[::7]  # بداية كل أسبوع

    for week_start in week_dates:
        used_teams = set()
        for home, away in pairings:
            if matches_count[(home, away)] >= 1:
                continue
            if home in used_teams or away in used_teams:
                continue

            possible_days = [d for d in all_dates if week_start <= d < week_start + timedelta(days=7)
                             and (d - last_played[home]).days >= min_rest_days
                             and (d - last_played[away]).days >= min_rest_days
                             and day_info[d]['count'] < 2]

            if not possible_days:
                continue

            # نحاول نختار وقت وملعب متاحين
            random.shuffle(possible_days)
            for date in possible_days:
                available_times = [t for t in match_times if t not in day_info[date]['times']]
                available_venues = [v for v in venues if v not in day_info[date]['venues']]
                if not available_times or not available_venues:
                    continue

                time = random.choice(available_times)
                venue = random.choice(available_venues)

                match = Match(home, away, date, time, venue)
                schedule.append(match)

                last_played[home] = date
                last_played[away] = date
                matches_count[(home, away)] += 1
                used_teams.update([home, away])

                day_info[date]['count'] += 1
                day_info[date]['venues'].add(venue)
                day_info[date]['times'].add(time)
                break  # بعد ما نحط المباراة نخرج

    schedule.sort(key=lambda m: m.date)
    return schedule

# طباعة جدول كل فريق
def print_team_schedules(schedule):
    team_schedule = {team: [] for team in teams}
    for match in schedule:
        team_schedule[match.team1].append(match)
        team_schedule[match.team2].append(match)
    for team, matches in team_schedule.items():
        print(f"\n=== Schedule for {team} ===")
        matches.sort(key=lambda m: m.date)
        for m in matches:
            opponent = m.team2 if m.team1 == team else m.team1
            print(f"{m.date.strftime('%Y-%m-%d')} at {m.time} vs {opponent} in {m.venue}")


# تجربة الكود
if __name__ == "__main__":
    schedule = generate_weekly_schedule(teams, venues, all_dates, match_times)
    print("=== First 20 Matches in Tournament ===")
    for m in schedule[:20]:
        print(m)
    print_team_schedules(schedule)


=== First 20 Matches in Tournament ===
El Dakhleya vs Zed on 2025-05-01 at 20:00 in Air Defense Stadium
Al Ahly vs Alexandria Union on 2025-05-02 at 17:00 in Suez Stadium
ENPPI vs Ismaily on 2025-05-02 at 20:00 in Borg El Arab
Zamalek vs Masry on 2025-05-03 at 20:00 in Suez Stadium
Wadi Degla vs Pharco on 2025-05-04 at 17:00 in Borg El Arab
Modern Sport vs El Gouna on 2025-05-04 at 20:00 in El Sekka El Hadeed Stadium
Smouha vs Pyramids on 2025-05-06 at 20:00 in Alexandria Stadium
Future vs Talaea El Gaish on 2025-05-06 at 17:00 in Al Salam Stadium
National Bank vs Ceramica on 2025-05-07 at 17:00 in El Sekka El Hadeed Stadium
Wadi Degla vs Ismaily on 2025-05-08 at 20:00 in Al Salam Stadium
El Gouna vs Ceramica on 2025-05-11 at 20:00 in Alexandria Stadium
Talaea El Gaish vs Alexandria Union on 2025-05-11 at 17:00 in Al Salam Stadium
Smouha vs Zed on 2025-05-12 at 17:00 in Suez Stadium
Future vs Zamalek on 2025-05-12 at 20:00 in Military Academy Stadium
National Bank vs Al Ahly on 2025-05

In [5]:
schedule = generate_weekly_schedule(teams, venues, all_dates, match_times)
schedule[:10]  # أول 10 مباريات


[Smouha vs Modern Sport on 2025-05-01 at 17:00 in El Sekka El Hadeed Stadium,
 Ceramica vs Al Ahly on 2025-05-01 at 20:00 in Alexandria Stadium,
 Pharco vs Masry on 2025-05-02 at 17:00 in Borg El Arab,
 El Dakhleya vs Zamalek on 2025-05-02 at 20:00 in Cairo Stadium,
 ENPPI vs Talaea El Gaish on 2025-05-05 at 17:00 in Air Defense Stadium,
 Ismaily vs National Bank on 2025-05-06 at 20:00 in Zed Club Stadium,
 Future vs El Gouna on 2025-05-06 at 17:00 in Borg El Arab,
 Alexandria Union vs Wadi Degla on 2025-05-07 at 17:00 in Military Academy Stadium,
 Zed vs Pyramids on 2025-05-07 at 20:00 in Al Salam Stadium,
 Al Ahly vs Modern Sport on 2025-05-10 at 17:00 in Al Salam Stadium]

In [6]:
print_team_schedules(schedule)



=== Schedule for Al Ahly ===
2025-05-01 at 20:00 vs Ceramica in Alexandria Stadium
2025-05-10 at 17:00 vs Modern Sport in Al Salam Stadium
2025-05-21 at 20:00 vs Masry in Borg El Arab
2025-05-25 at 20:00 vs Pyramids in Borg El Arab
2025-05-31 at 17:00 vs National Bank in Zed Club Stadium
2025-06-09 at 20:00 vs El Gouna in Borg El Arab
2025-06-14 at 20:00 vs Smouha in Suez Stadium
2025-06-24 at 17:00 vs Ismaily in Military Academy Stadium
2025-07-02 at 17:00 vs Alexandria Union in Cairo Stadium
2025-07-07 at 20:00 vs Talaea El Gaish in Military Academy Stadium
2025-07-11 at 20:00 vs Zamalek in Borg El Arab
2025-07-23 at 17:00 vs Alexandria Union in Al Salam Stadium
2025-07-29 at 20:00 vs Ismaily in El Sekka El Hadeed Stadium
2025-08-05 at 20:00 vs National Bank in Cairo Stadium
2025-08-13 at 20:00 vs Pyramids in Suez Stadium
2025-08-18 at 20:00 vs Modern Sport in Military Academy Stadium
2025-08-27 at 20:00 vs El Gouna in El Sekka El Hadeed Stadium
2025-09-01 at 17:00 vs ENPPI in Borg 

In [1]:
!pip install ipywidgets


   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/914.9 kB ? eta -:--:--
   ---------------------------------- ----- 786.4/914.9 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 914.9/914.9 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 2.9 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.2 MB 2.9 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 2.6 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.2 MB 2.4 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 2.2 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 1.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# ===== واجهة المستخدم الجديدة =====
import ipywidgets as widgets
from IPython.display import display, HTML

view_dropdown = widgets.Dropdown(
    options=["Full League Schedule", "Team Schedule"],
    description="View:",
    disabled=False,
)

team_dropdown = widgets.Dropdown(
    options=teams,
    description="Team:",
    disabled=False,
)

button = widgets.Button(description="Show Schedule")
output = widgets.Output()

def show_schedule(b):
    output.clear_output()
    view = view_dropdown.value
    with output:
        if view == "Full League Schedule":
            html = "<h2>Full League Schedule</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Home</th><th>Away</th><th>Venue</th></tr>"
            for match in full_schedule:
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{match.team1}</td><td>{match.team2}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))
        else:
            team = team_dropdown.value
            team_matches = [m for m in full_schedule if m.team1 == team or m.team2 == team]
            html = f"<h2>Schedule for {team}</h2><table border='1' style='border-collapse: collapse;'>"
            html += "<tr><th>Date</th><th>Time</th><th>Opponent</th><th>Venue</th></tr>"
            for match in team_matches:
                opponent = match.team2 if match.team1 == team else match.team1
                html += f"<tr><td>{match.date.strftime('%Y-%m-%d')}</td><td>{match.time}</td><td>{opponent}</td><td>{match.venue}</td></tr>"
            html += "</table>"
            display(HTML(html))

button.on_click(show_schedule)
display(view_dropdown, team_dropdown, button, output)


Dropdown(description='View:', options=('Full League Schedule', 'Team Schedule'), value='Full League Schedule')

Dropdown(description='Team:', options=('Al Ahly', 'Zamalek', 'Pyramids', 'Masry', 'Future', 'Ismaily', 'Smouha…

Button(description='Show Schedule', style=ButtonStyle())

Output()